In [1]:
import numpy as np
import networkx as nx
import json

In [ ]:
import heapq

def dijkstra(graph, start_node):
    """
    Dijkstra's algorithm to find the shortest paths from a single source node.

    Args:
        graph (dict): A dictionary representing the graph where keys are nodes
                      and values are lists of tuples (neighbor, weight).
        start_node: The starting node for the algorithm.

    Returns:
        dict: A dictionary containing the shortest distances from the start node
              to all other nodes in the graph.
    """
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0
    priority_queue = [(0, start_node)]  # (distance, node)
    visited = set()

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node in visited:
            continue
        visited.add(current_node)

        for neighbor, weight in graph.get(current_node, []):
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

# Example graph from the previous explanation
example_graph = {
    'A': [('B', 2), ('A', 3)],
    'B': [('C', 5), ('D', 1)],
    'C': [('D', 4)],
    'D': [('A', 2)]
}

# Apply Dijkstra's algorithm starting from node 'A'
start_node = 'A'
shortest_distances = dijkstra(example_graph, start_node)

# Print the results
print(f"Shortest distances from node '{start_node}':")
for node, distance in shortest_distances.items():
    print(f"To node {node}: {distance}")

Shortest distances from node 'A':
To node A: 0
To node B: 2
To node C: 7
To node D: 3


In [ ]:
def dfs_paths_of_length(graph, start_node, length):
    """
    Finds all paths of a specified length in a directed and weighted graph using Depth-First Search (DFS).

    Args:
        graph: A dictionary representing the graph where keys are nodes and values are lists of (neighbor, weight) tuples.
        start_node: The starting node for the paths.
        length: The desired length of the paths.

    Returns:
        A list of paths, where each path is a list of nodes.  Returns an empty list if no paths of the specified length are found.
    """

    paths = []

    def dfs(current_node, current_path, current_length):
        """
        Recursive DFS helper function.
        """
        if current_length == length:
            paths.append(current_path.copy())  # Append a copy to avoid modification issues
            return

        if current_length > length:
            return  # Prune paths that are already too long

        if current_node not in graph:
            return  # Dead end: node has no outgoing edges

        for neighbor, _ in graph[current_node]:  # Iterate through neighbors (ignore weight for path finding)
            current_path.append(neighbor)
            dfs(neighbor, current_path, current_length + 1)
            current_path.pop()  # Backtrack: remove the neighbor to explore other paths

    dfs(start_node, [start_node], 0)
    return paths


# Example Usage and Testing
if __name__ == "__main__":
    # Example Graph (Directed and Weighted)
    graph = {
    'A': [('B', 2), ('A', 3)],
    'B': [('C', 5), ('D', 1)],
    'C': [('D', 4)],
    'D': [('A', 2)]
}

    start_node = 'A'
    path_length = 3

    paths = dfs_paths_of_length(graph, start_node, path_length)

    print(f"Paths of length {path_length} starting from node {start_node}:")
    if paths:
        for path in paths:
            print(path)
    else:
        print("No paths found of the specified length.")

    # Test case 2:  Different start node and length
    start_node = 'B'
    path_length = 2
    paths = dfs_paths_of_length(graph, start_node, path_length)
    print(f"\nPaths of length {path_length} starting from node {start_node}:")
    if paths:
        for path in paths:
            print(path)
    else:
        print("No paths found of the specified length.")

Paths of length 3 starting from node A:
['A', 'B', 'C', 'D']
['A', 'B', 'D', 'A']
['A', 'A', 'B', 'C']
['A', 'A', 'B', 'D']
['A', 'A', 'A', 'B']
['A', 'A', 'A', 'A']

Paths of length 2 starting from node B:
['B', 'C', 'D']
['B', 'D', 'A']


In [ ]:
class Cube:
    def __init__(self):
        self.piece_current_positions = np.array([
                    [[1 , 2  , 3 ], [4  , 'F', 6  ], [7 , 8  , 9 ]],
                    [[10, 'U', 12], ['L', 'C', 'R'], [16, 'D', 18]],
                    [[19, 20 , 21], [22 , 'B', 24 ], [25, 26 , 27]]
                ]) #The PIECES are counted, not the colors, from Left-to-Right, Top-to-Bottom, and Front-to-Back. The fourteenth piece is the invisible and irrelevant center piece of the cube, but is there for mathematical simplicity
        self.piece_current_orientations = np.array([
            [['x', 'g', 'x'], ['g', 'F', 'g'], ['x', 'g', 'x']],
            [['g', 'U', 'g'], ['L', 'C', 'R'], ['g', 'D', 'g']],
            [['x', 'g', 'x'], ['g', 'B', 'g'], ['x', 'g', 'x']],
        ])
        #notice that only the corners are indicated by 'x' (other possible values are 'y' and 'z') and only the edges are indicated by 'g' ('g' for good, other possible value is 'b' for bad)
        #here x stands for the x-axis, and it means that the corner's fixed x-axis (the axis rotates when the corner rotates) is aligned with the cube's x-axis
        #g stands for "good", which is one of the two possible orientations of an edge piece at any given position
        #and other one is b, which stands for "bad"
        #the center pieces are not indicated, as they are irrelevant to the orientation of the pieces of the cube
        self.move_map = {
            'U': self.U,
            'F': self.F,
            'B': self.B,
            'D': self.D,
            'L': self.L,
            'R': self.R,
            'u': self.u,
            'f': self.f,
            'b': self.b,
            'd': self.d,
            'l': self.l,
            'r': self.r,
            'M': self.M,
            'E': self.E,
            'S': self.S,
            'm': self.m,
            'e': self.e,
            's': self.s,
        }
    def _rotate_slice(self, perspective, slice_idx, direction):
        """Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction"""
        # Convert to the desired perspective, rotate the slice, then convert back
        def change_perspective(cube, perspective, direction):
            if perspective == 0:
                return cube
            else:
                return np.rot90(cube, k=direction, axes=(0, perspective))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        # Rotate the slice
        self.piece_current_positions[slice_idx] = np.rot90(self.piece_current_positions[slice_idx], k=direction, axes=(0, 1))
        # Convert back to the original perspective
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)
        
    def F(self): self._rotate_slice(perspective=0, slice_idx=0, direction=-1)
    def f(self): self._rotate_slice(perspective=0, slice_idx=0, direction=1)
    def M(self): self._rotate_slice(perspective=0, slice_idx=1, direction=-1)
    def m(self): self._rotate_slice(perspective=0, slice_idx=1, direction=1)
    def B(self): self._rotate_slice(perspective=0, slice_idx=2, direction=1)
    def b(self): self._rotate_slice(perspective=0, slice_idx=2, direction=-1)
    def U(self): self._rotate_slice(perspective=1, slice_idx=0, direction=-1)
    def u(self): self._rotate_slice(perspective=1, slice_idx=0, direction=1)
    def E(self): self._rotate_slice(perspective=1, slice_idx=1, direction=-1)
    def e(self): self._rotate_slice(perspective=1, slice_idx=1, direction=1)
    def D(self): self._rotate_slice(perspective=1, slice_idx=2, direction=1)
    def d(self): self._rotate_slice(perspective=1, slice_idx=2, direction=-1)
    def L(self): self._rotate_slice(perspective=2, slice_idx=0, direction=-1)
    def l(self): self._rotate_slice(perspective=2, slice_idx=0, direction=1)
    def S(self): self._rotate_slice(perspective=2, slice_idx=1, direction=-1)
    def s(self): self._rotate_slice(perspective=2, slice_idx=1, direction=1)
    def R(self): self._rotate_slice(perspective=2, slice_idx=2, direction=1)
    def r(self): self._rotate_slice(perspective=2, slice_idx=2, direction=-1)

    def apply_moves(self, move_sequence):
        for move in move_sequence:
                if move in self.move_map:
                    self.move_map[move]()
                else:
                    raise ValueError(f"Invalid move: {move}")

In [ ]:
c = Cube()
#c.apply_moves("b")  # Apply the scramble moves to the cube
c.apply_moves("ldBUUBDUUlFLL")  # Apply the scramble moves to the cube
print(c.piece_current_positions)

[[['21' '20' '3']
  ['4' 'F' '22']
  ['25' '6' '7']]

 [['2' 'U' '12']
  ['L' 'C' 'R']
  ['16' 'D' '18']]

 [['9' '24' '19']
  ['8' 'B' '10']
  ['1' '26' '27']]]


In [ ]:
import numpy as np
import json
import copy

def generate_move_table():
    """Generate a table of how each move affects piece positions"""
    cube = Cube()  # Create a fresh cube in solved state
    movement_table = {}
    
    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube
        test_cube = Cube()
        
        # Record initial positions of all pieces
        initial_positions = {}
        for piece_id in range(1, 28):  # All piece IDs
            if piece_id not in [5, 11, 13, 14, 15, 17, 23]:  # Skip center piece
                pos = {list(zip(*np.where(test_cube == piece_id)))}
                if pos:
                    initial_positions[piece_id] = pos
        
        # Apply the move
        getattr(test_cube, move)()
        
        # Record final positions and calculate movements
        movements = {}
        for piece_id, initial_pos in initial_positions.items():
            final_pos = test_cube._get_position_of_piece(piece_id)
            if initial_pos != final_pos:
                movements[piece_id] = {
                    "from": initial_pos,
                    "to": final_pos
                }
        
        # Store in table
        movement_table[move] = movements
    
    # Save to file
    with open('rubik_moves_table.json', 'w') as f:
        # Convert tuple keys to strings for JSON serialization
        serializable_table = {}
        for move, piece_movements in movement_table.items():
            serializable_movements = {}
            for piece_id, movement in piece_movements.items():
                from_pos = ','.join(map(str, movement["from"]))
                to_pos = ','.join(map(str, movement["to"]))
                serializable_movements[piece_id] = {
                    "from": from_pos,
                    "to": to_pos
                }
            serializable_table[move] = serializable_movements
        
        json.dump(serializable_table, f, indent=2)
    
    return movement_table

# Example function to load the table
def load_move_table():
    """Load the pre-generated move table"""
    with open('rubik_moves_table.json', 'r') as f:
        serialized_table = json.load(f)
        
    # Convert string positions back to tuples
    movement_table = {}
    for move, piece_movements in serialized_table.items():
        movements = {}
        for piece_id_str, movement in piece_movements.items():
            piece_id = int(piece_id_str)
            from_pos = tuple(map(int, movement["from"].split(',')))
            to_pos = tuple(map(int, movement["to"].split(',')))
            movements[piece_id] = {
                "from": from_pos,
                "to": to_pos
            }
        movement_table[move] = movements
    
    return movement_table

In [27]:
a = 'c'
type(a) #int

str

In [ ]:
# Create a sample array
arr = np.array([1, 2, 3, 4, 5, 2, 7])

# Method 1: Using np.where()
indices = np.where(arr == 2)
print(f"Indices of element 2 using np.where(): {indices[0]}")

# Method 2: Using np.argwhere()
indices2 = np.argwhere(arr == 2)
print(f"Indices of element 2 using np.argwhere(): {indices2.flatten()}")

# For multi-dimensional arrays (like in the cube)
multi_arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("\nMulti-dimensional array:")
print(multi_arr)

# Find the index of element 5
element_indices = np.where(multi_arr == 5)
print(f"\nIndices of element 5 using np.where(): {list(zip(*element_indices))}")

# Alternative with argwhere
element_indices2 = np.argwhere(multi_arr == 5)
print(f"Indices of element 5 using np.argwhere(): {element_indices2}")

# Example with the cube's piece_current_positions
piece_indices = np.where(c.piece_current_positions == 7)
print(f"\nIndices of piece 7 in the cube: {list(zip(*piece_indices))}")

Indices of element 2 using np.where(): [1 5]
Indices of element 2 using np.argwhere(): [1 5]

Multi-dimensional array:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Indices of element 5 using np.where(): [(1, 1)]
Indices of element 5 using np.argwhere(): [[1 1]]

Indices of piece 7 in the cube: []


In [ ]:
def get_position_of_piece(cube, piece_id):
    

# Create a fresh cube in the solved state
test_cube = Cube()

# Record initial positions of all pieces
initial_positions = {}
for piece_id in range(1, 28):  # All piece IDs
    # Skip the center pieces (actual centers, not just piece 14)
    if piece_id not in [5, 11, 13, 14, 15, 17, 23]:
        pos = get_position_of_piece(test_cube, piece_id)
        if pos:
            initial_positions[piece_id] = pos
        else:
            print(f"Warning: Piece {piece_id} not found in the cube")

print(initial_positions)

{}


In [ ]:
import numpy as np
"""Find the position of a specific piece in the cube"""
initial_positions = {}
c = Cube()
for i in range(3):
    for j in range(3):
        for k in range(3):
            value = c.piece_current_positions[i, j, k]
            # Handle both integer and string comparison
            if isinstance(value, (int, np.integer)):
                initial_positions[value] = (i, j, k)
print(initial_positions)

{}


In [ ]:
class Cube:
    def __init__(self):
        self.piece_current_positions = np.array([
                    [[1 , 2 , 3 ], [4 , 5 , 6 ], [7 , 8 , 9 ]],
                    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
                    [[19, 20, 21], [22, 23, 24], [25, 26 , 27]]
                ]) #The PIECES are counted, not the colors, from Left-to-Right, Top-to-Bottom, and Front-to-Back. The fourteenth piece is the invisible and irrelevant center piece of the cube, but is there for mathematical simplicity
        self.piece_current_orientations = np.array([
            [['x', 'g', 'x'], ['g', 'F', 'g'], ['x', 'g', 'x']],
            [['g', 'U', 'g'], ['L', 'C', 'R'], ['g', 'D', 'g']],
            [['x', 'g', 'x'], ['g', 'B', 'g'], ['x', 'g', 'x']],
        ])
        #notice that only the corners are indicated by 'x' (other possible values are 'y' and 'z') and only the edges are indicated by 'g' ('g' for good, other possible value is 'b' for bad)
        #here x stands for the x-axis, and it means that the corner's fixed x-axis (the axis rotates when the corner rotates) is aligned with the cube's x-axis
        #g stands for "good", which is one of the two possible orientations of an edge piece at any given position
        #and other one is b, which stands for "bad"
        #the center pieces are not indicated, as they are irrelevant to the orientation of the pieces of the cube
        self.move_map = {
            'U': self.U,
            'F': self.F,
            'B': self.B,
            'D': self.D,
            'L': self.L,
            'R': self.R,
            'u': self.u,
            'f': self.f,
            'b': self.b,
            'd': self.d,
            'l': self.l,
            'r': self.r,
            'M': self.M,
            'E': self.E,
            'S': self.S,
            'm': self.m,
            'e': self.e,
            's': self.s,
        }
    def _rotate_slice(self, perspective, slice_idx, direction):
        """Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction"""
        # Convert to the desired perspective, rotate the slice, then convert back
        def change_perspective(cube, perspective, direction):
            if perspective == 0:
                return cube
            else:
                return np.rot90(cube, k=direction, axes=(0, perspective))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        # Rotate the slice
        self.piece_current_positions[slice_idx] = np.rot90(self.piece_current_positions[slice_idx], k=direction, axes=(0, 1))
        # Convert back to the original perspective
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)
        
    def F(self): self._rotate_slice(perspective=0, slice_idx=0, direction=-1)
    def f(self): self._rotate_slice(perspective=0, slice_idx=0, direction=1)
    def M(self): self._rotate_slice(perspective=0, slice_idx=1, direction=-1)
    def m(self): self._rotate_slice(perspective=0, slice_idx=1, direction=1)
    def B(self): self._rotate_slice(perspective=0, slice_idx=2, direction=1)
    def b(self): self._rotate_slice(perspective=0, slice_idx=2, direction=-1)
    def U(self): self._rotate_slice(perspective=1, slice_idx=0, direction=-1)
    def u(self): self._rotate_slice(perspective=1, slice_idx=0, direction=1)
    def E(self): self._rotate_slice(perspective=1, slice_idx=1, direction=-1)
    def e(self): self._rotate_slice(perspective=1, slice_idx=1, direction=1)
    def D(self): self._rotate_slice(perspective=1, slice_idx=2, direction=1)
    def d(self): self._rotate_slice(perspective=1, slice_idx=2, direction=-1)
    def L(self): self._rotate_slice(perspective=2, slice_idx=0, direction=-1)
    def l(self): self._rotate_slice(perspective=2, slice_idx=0, direction=1)
    def S(self): self._rotate_slice(perspective=2, slice_idx=1, direction=-1)
    def s(self): self._rotate_slice(perspective=2, slice_idx=1, direction=1)
    def R(self): self._rotate_slice(perspective=2, slice_idx=2, direction=1)
    def r(self): self._rotate_slice(perspective=2, slice_idx=2, direction=-1)

    def apply_moves(self, move_sequence):
        for move in move_sequence:
                if move in self.move_map:
                    self.move_map[move]()
                else:
                    raise ValueError(f"Invalid move: {move}")

In [ ]:
import numpy as np
"""Find the position of a specific piece in the cube"""
initial_positions = {}
c = Cube()
for i in range(3):
    for j in range(3):
        for k in range(3):
            value = c.piece_current_positions[i, j, k]
            # Handle both integer and string comparison
            if value not in [5, 11, 13, 14, 15, 17, 23]:
                initial_positions[value] = (i, j, k)
print(initial_positions)

{1: (0, 0, 0), 2: (0, 0, 1), 3: (0, 0, 2), 4: (0, 1, 0), 6: (0, 1, 2), 7: (0, 2, 0), 8: (0, 2, 1), 9: (0, 2, 2), 10: (1, 0, 0), 12: (1, 0, 2), 16: (1, 2, 0), 18: (1, 2, 2), 19: (2, 0, 0), 20: (2, 0, 1), 21: (2, 0, 2), 22: (2, 1, 0), 24: (2, 1, 2), 25: (2, 2, 0), 26: (2, 2, 1), 27: (2, 2, 2)}


In [ ]:
import numpy as np
import json
import copy

def generate_move_table():
    """Generate a table of how each move affects piece positions"""
    cube = Cube()  # Create a fresh cube in solved state
    movement_table = {}
    
    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube
        test_cube = Cube()
        
        # Record initial positions of all pieces
        """Find the position of a specific piece in the cube"""
        initial_positions = {}
        c = Cube()
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    value = c.piece_current_positions[i, j, k]
                    # Handle both integer and string comparison
                    if value not in [5, 11, 13, 14, 15, 17, 23]:
                        initial_positions[value] = (i, j, k)
        
        # Apply the move
        getattr(test_cube, move)()
        
        # Record final positions and calculate movements
        movements = {}
        for piece_id, initial_pos in initial_positions.items():
            final_pos = test_cube._get_position_of_piece(piece_id)
            if initial_pos != final_pos:
                movements[piece_id] = {
                    "from": initial_pos,
                    "to": final_pos
                }
        
        # Store in table
        movement_table[move] = movements
    
    # Save to file
    with open('rubik_moves_table.json', 'w') as f:
        # Convert tuple keys to strings for JSON serialization
        serializable_table = {}
        for move, piece_movements in movement_table.items():
            serializable_movements = {}
            for piece_id, movement in piece_movements.items():
                from_pos = ','.join(map(str, movement["from"]))
                to_pos = ','.join(map(str, movement["to"]))
                serializable_movements[piece_id] = {
                    "from": from_pos,
                    "to": to_pos
                }
            serializable_table[move] = serializable_movements
        
        json.dump(serializable_table, f, indent=2)
    
    return movement_table

# Example function to load the table
def load_move_table():
    """Load the pre-generated move table"""
    with open('rubik_moves_table.json', 'r') as f:
        serialized_table = json.load(f)
        
    # Convert string positions back to tuples
    movement_table = {}
    for move, piece_movements in serialized_table.items():
        movements = {}
        for piece_id_str, movement in piece_movements.items():
            piece_id = int(piece_id_str)
            from_pos = tuple(map(int, movement["from"].split(',')))
            to_pos = tuple(map(int, movement["to"].split(',')))
            movements[piece_id] = {
                "from": from_pos,
                "to": to_pos
            }
        movement_table[move] = movements
    
    return movement_table

In [ ]:
from collections import deque
import numpy as np
import json
import copy

class Cube:
    def __init__(self):
        self.piece_current_positions = np.array([
                    [[1 , 2 , 3 ], [4 , 5 , 6 ], [7 , 8 , 9 ]],
                    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
                    [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
                ]) #The PIECES are counted, not the colors, from Left-to-Right, Top-to-Bottom, and Front-to-Back. The fourteenth piece is the invisible and irrelevant center piece of the cube, but is there for mathematical simplicity
        self.piece_current_orientations = np.array([
            [['x', 'g', 'x'], ['g', 'F', 'g'], ['x', 'g', 'x']],
            [['g', 'U', 'g'], ['L', 'C', 'R'], ['g', 'D', 'g']],
            [['x', 'g', 'x'], ['g', 'B', 'g'], ['x', 'g', 'x']],
        ])
        #notice that only the corners are indicated by 'x' (other possible values are 'y' and 'z') and only the edges are indicated by 'g' ('g' for good, other possible values are 'b' for bad)
        #here x stands for the x-axis, and it means that the corner's fixed x-axis (the axis rotates when the corner rotates) is aligned with the cube's x-axis
        #g stands for "good", which is one of the two possible orientations of an edge piece at any given position
        #and other one is b, which stands for "bad"
        #the center pieces are not indicated, as they always remain the same
        self.move_map = {
            'U': self.U,
            'F': self.F,
            'B': self.B,
            'D': self.D,
            'L': self.L,
            'R': self.R,
            'u': self.u,
            'f': self.f,
            'b': self.b,
            'd': self.d,
            'l': self.l,
            'r': self.r,
            'M': self.M,
            'E': self.E,
            'S': self.S,
            'm': self.m,
            'e': self.e,
            's': self.s,
        }
    def _rotate_slice(self, perspective, slice_idx, direction):
        """Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction"""
        # Convert to the desired perspective, rotate the slice, then convert back
        def change_perspective(cube, perspective, direction):
            if perspective == 0:
                return cube
            else:
                return np.rot90(cube, k=direction, axes=(0, perspective))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        # Rotate the slice
        self.piece_current_positions[slice_idx] = np.rot90(self.piece_current_positions[slice_idx], k=direction, axes=(0, 1))
        # Convert back to the original perspective
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)

    def _get_position_of_piece(self, piece_id):
        """Find the 3D position (index) of a piece given its ID."""
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_positions[i, j, k] == piece_id:
                        return (i, j, k)
        return None  # Piece ID not found (should not happen in a valid cube state)

    def F(self): self._rotate_slice(perspective=0, slice_idx=0, direction=-1)
    def f(self): self._rotate_slice(perspective=0, slice_idx=0, direction=1)
    def M(self): self._rotate_slice(perspective=0, slice_idx=1, direction=-1)
    def m(self): self._rotate_slice(perspective=0, slice_idx=1, direction=1)
    def B(self): self._rotate_slice(perspective=0, slice_idx=2, direction=1)
    def b(self): self._rotate_slice(perspective=0, slice_idx=2, direction=-1)
    def U(self): self._rotate_slice(perspective=1, slice_idx=0, direction=-1)
    def u(self): self._rotate_slice(perspective=1, slice_idx=0, direction=1)
    def E(self): self._rotate_slice(perspective=1, slice_idx=1, direction=-1)
    def e(self): self._rotate_slice(perspective=1, slice_idx=1, direction=1)
    def D(self): self._rotate_slice(perspective=1, slice_idx=2, direction=1)
    def d(self): self._rotate_slice(perspective=1, slice_idx=2, direction=-1)
    def L(self): self._rotate_slice(perspective=2, slice_idx=0, direction=-1)
    def l(self): self._rotate_slice(perspective=2, slice_idx=0, direction=1)
    def S(self): self._rotate_slice(perspective=2, slice_idx=1, direction=-1)
    def s(self): self._rotate_slice(perspective=2, slice_idx=1, direction=1)
    def R(self): self._rotate_slice(perspective=2, slice_idx=2, direction=1)
    def r(self): self._rotate_slice(perspective=2, slice_idx=2, direction=-1)

    def apply_moves(self, move_sequence):
        for move in move_sequence:
                if move in self.move_map:
                    self.move_map[move]()
                else:
                    raise ValueError(f"Invalid move: {move}")

def generate_move_table():
    """
    Generates a table mapping each move to the pieces it moves and their position changes.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              describing piece movements.
    """
    cube = Cube()  # Create a fresh cube in solved state
    movement_table = {}

    # Pieces that are not centers and thus their positions are tracked
    tracked_pieces = set(range(1, 28)) - {5, 11, 13, 14, 15, 17, 23}

    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube for each move calculation
        test_cube = Cube()

        # Record initial positions of tracked pieces
        initial_positions = {}
        for piece_id in tracked_pieces:
            initial_positions[piece_id] = test_cube._get_position_of_piece(piece_id)

        # Apply the move
        getattr(test_cube, move)()

        # Record final positions and calculate movements
        movements = {}
        for piece_id in tracked_pieces:
            initial_pos = initial_positions[piece_id]
            final_pos = test_cube._get_position_of_piece(piece_id)
            if initial_pos != final_pos:
                movements[piece_id] = {
                    "from": initial_pos,
                    "to": final_pos
                }

        # Store in table
        movement_table[move] = movements

    # Save to file
    with open('rubik_moves_table.json', 'w') as f:
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, piece_movements in movement_table.items():
            serializable_movements = {}
            for piece_id, movement in piece_movements.items():
                from_pos = ','.join(map(str, movement["from"]))
                to_pos = ','.join(map(str, movement["to"]))
                serializable_movements[piece_id] = {
                    "from": from_pos,
                    "to": to_pos
                }
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)

    return movement_table

# Example function to load the table
def load_move_table():
    """
    Loads the pre-generated move table from 'rubik_moves_table.json'.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              describing piece movements.
    """
    try:
        with open('rubik_moves_table.json', 'r') as f:
            serialized_table = json.load(f)
    except FileNotFoundError:
        print("Error: 'rubik_moves_table.json' not found. Please generate it using 'generate_move_table()'.")
        return None

    # Convert string positions back to tuples
    movement_table = {}
    for move, piece_movements in serialized_table.items():
        movements = {}
        for piece_id_str, movement in piece_movements.items():
            piece_id = int(piece_id_str)
            from_pos = tuple(map(int, movement["from"].split(',')))
            to_pos = tuple(map(int, movement["to"].split(',')))
            movements[piece_id] = {
                "from": from_pos,
                "to": to_pos
            }
        movement_table[move] = movements

    return movement_table
generate_move_table()
load_move_table()


{'U': {1: {'from': (0, 0, 0), 'to': (2, 0, 0)},
  2: {'from': (0, 0, 1), 'to': (1, 0, 0)},
  3: {'from': (0, 0, 2), 'to': (0, 0, 0)},
  10: {'from': (1, 0, 0), 'to': (2, 0, 1)},
  12: {'from': (1, 0, 2), 'to': (0, 0, 1)},
  19: {'from': (2, 0, 0), 'to': (2, 0, 2)},
  20: {'from': (2, 0, 1), 'to': (1, 0, 2)},
  21: {'from': (2, 0, 2), 'to': (0, 0, 2)}},
 'F': {1: {'from': (0, 0, 0), 'to': (0, 0, 2)},
  2: {'from': (0, 0, 1), 'to': (0, 1, 2)},
  3: {'from': (0, 0, 2), 'to': (0, 2, 2)},
  4: {'from': (0, 1, 0), 'to': (0, 0, 1)},
  6: {'from': (0, 1, 2), 'to': (0, 2, 1)},
  7: {'from': (0, 2, 0), 'to': (0, 0, 0)},
  8: {'from': (0, 2, 1), 'to': (0, 1, 0)},
  9: {'from': (0, 2, 2), 'to': (0, 2, 0)}},
 'B': {19: {'from': (2, 0, 0), 'to': (2, 2, 0)},
  20: {'from': (2, 0, 1), 'to': (2, 1, 0)},
  21: {'from': (2, 0, 2), 'to': (2, 0, 0)},
  22: {'from': (2, 1, 0), 'to': (2, 2, 1)},
  24: {'from': (2, 1, 2), 'to': (2, 0, 1)},
  25: {'from': (2, 2, 0), 'to': (2, 2, 2)},
  26: {'from': (2, 2, 1), 

In [ ]:
def get_piece_types(cube):
    """Identifies edge and corner piece IDs based on orientation markers."""
    corner_pieces = []
    edge_pieces = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                piece_id = cube.piece_current_positions[i, j, k]
                orientation = cube.piece_current_orientations[i, j, k]
                if orientation == 'x':  # Corner piece
                    corner_pieces.append(piece_id)
                elif orientation == 'g':  # Edge piece
                    edge_pieces.append(piece_id)
    return corner_pieces, edge_pieces
def calculate_distance_table(piece_ids, all_moves):
    """
    Calculates the distance table between pieces of the same type using BFS.

    Args:
        piece_ids (list): List of piece IDs to calculate distances for.
        all_moves (list): List of valid moves (strings) for the cube.

    Returns:
        dict: A dictionary representing the distance table where keys are piece ID pairs
              (tuples) and values are the minimum distances.
    """
    distance_table = {}
    solved_cube = Cube() # create a solved cube instance here to be used in the function


    for piece_id_start in piece_ids:
        for piece_id_target in piece_ids:
            if piece_id_start == piece_id_target:
                distance_table[(piece_id_start, piece_id_target)] = 0
                continue # Distance to itself is 0

            if (piece_id_target, piece_id_start) in distance_table: #already computed
                distance_table[(piece_id_start, piece_id_target)] = distance_table[(piece_id_target, piece_id_start)]
                continue

            start_pos = solved_cube._get_position_of_piece(piece_id_start) #initial position of starting piece in solved cube
            target_start_pos = solved_cube._get_position_of_piece(piece_id_target) #initial position of target piece in solved cube


            queue = deque([(Cube(), 0)])  # Queue of (cube_state, distance)
            visited_states = {cube_to_tuple(Cube())}  # Set of visited cube states (represented as tuples)

            distance = -1 # Initialize distance to -1 (not found yet)

            while queue:
                current_cube, current_distance = queue.popleft()

                # Check if target piece is at the starting position
                target_current_pos = current_cube._get_position_of_piece(piece_id_target)
                if target_current_pos == start_pos:
                    distance = current_distance
                    break # Found the shortest path


                if current_distance >= 15: #Optimization: max distance in cube is 20, for piece position likely much smaller, stop searching at distance 15 for faster computation
                    continue

                for move in all_moves:
                    next_cube = copy.deepcopy(current_cube) # Create a copy to avoid modifying the original cube in queue
                    next_cube.move_map[move]() # Apply the move
                    next_state_tuple = cube_to_tuple(next_cube)

                    if next_state_tuple not in visited_states:
                        visited_states.add(next_state_tuple)
                        queue.append((next_cube, current_distance + 1))
            distance_table[(piece_id_start, piece_id_target)] = distance

    return distance_table

def cube_to_tuple(cube):
    """Converts the cube's piece_current_positions NumPy array to a tuple for hashability."""
    return tuple(cube.piece_current_positions.flatten().tolist())


def save_distance_table_to_json(distance_table, filename):
    """Saves the distance table to a JSON file."""
    serializable_table = {}
    for piece_pair, distance in distance_table.items():
        serializable_table[str(piece_pair)] = distance #json requires string keys
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)

# Get the piece types from a solved cube
solved_cube = Cube()
corner_piece_ids, edge_piece_ids = get_piece_types(solved_cube)

# Get all possible moves from the move_map
all_moves = list(solved_cube.move_map.keys())


# Calculate distance tables
edge_distance_table = calculate_distance_table(edge_piece_ids, all_moves)
corner_distance_table = calculate_distance_table(corner_piece_ids, all_moves)


save_distance_table_to_json(edge_distance_table, "edge_distance_table.json")
save_distance_table_to_json(corner_distance_table, "corner_distance_table.json")


print("Edge Distance Table (example pairs):")
edge_example_pairs = [(2, 4), (2, 6), (2,26), (10, 26)] # Example pairs
for pair in edge_example_pairs:
    print(f"Distance between edge pieces {pair[0]} and {pair[1]}: {edge_distance_table[pair]}")

print("\nCorner Distance Table (example pairs):")
corner_example_pairs = [(1, 3), (1, 7), (1, 27), (9, 27)] # Example pairs
for pair in corner_example_pairs:
    print(f"Distance between corner pieces {pair[0]} and {pair[1]}: {corner_distance_table[pair]}")

Edge Distance Table (example pairs):
Distance between edge pieces 2 and 4: 1
Distance between edge pieces 2 and 6: 1
Distance between edge pieces 2 and 26: 2
Distance between edge pieces 10 and 26: 2

Corner Distance Table (example pairs):
Distance between corner pieces 1 and 3: 1
Distance between corner pieces 1 and 7: 1
Distance between corner pieces 1 and 27: 3
Distance between corner pieces 9 and 27: 1


In [ ]:
from collections import deque
import numpy as np
import json
import copy

class Cube:
    def __init__(self):
        self.piece_current_positions = np.array([
                    [[1 , 2 , 3 ], [4 , 5 , 6 ], [7 , 8 , 9 ]],
                    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
                    [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
                ])
        #The PIECES are counted from Left-to-Right(axis=2), Top-to-Bottom (axis=1), and Front-to-Back (axis=0), in that order. The fourteenth piece is the invisible and irrelevant center-most piece of the cube

        self.piece_current_orientations = np.array([
            [['x', 'g', 'x'], ['g', 'F', 'g'], ['x', 'g', 'x']],
            [['g', 'U', 'g'], ['L', 'C', 'R'], ['g', 'D', 'g']],
            [['x', 'g', 'x'], ['g', 'B', 'g'], ['x', 'g', 'x']],
        ])
        # This is the intial state of the cube, which is a solved state
        # Notice that only the corners are marked by 'x' (other possible values are 'y' and 'z') and only the edges are marked by 'g' ('g' for good, other possible value is 'b' for bad)
        # Here x stands for the x-axis, and it means that the corner's fixed x-axis is aligned with the cube's x-axis (when the corner rotates, its axes rotate too) 
        # g stands for "good", which is one of the two possible orientations of an edge piece at any given position, and other one is b, which stands for "bad"
        # The center pieces are not marked, as they always remain the same

        self.move_map = {
            'U': self.U, 'F': self.F, 'B': self.B, 'D': self.D, 'L': self.L, 'R': self.R,
            'u': self.u, 'f': self.f, 'b': self.b, 'd': self.d, 'l': self.l, 'r': self.r,
            'M': self.M, 'E': self.E, 'S': self.S, 'm': self.m, 'e': self.e, 's': self.s,
        }

    def _rotate_slice(self, perspective, slice_idx, direction):
        # Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction
        # Convert to the desired perspective, rotate the slice, then convert back
        def change_perspective(cube, perspective, direction):
            if perspective == 0: return cube
            else: return np.rot90(cube, k=direction, axes=(0, perspective))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        self.piece_current_positions[slice_idx] = np.rot90(self.piece_current_positions[slice_idx], k=direction, axes=(0, 1))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)

    def _get_position_of_piece(self, piece_id):
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_positions[i, j, k] == piece_id:
                        return (i, j, k)
        return None # Piece ID not found (should not happen in a valid cube state)
    
    def F(self): self._rotate_slice(perspective=0, slice_idx=0, direction=-1)
    def f(self): self._rotate_slice(perspective=0, slice_idx=0, direction=1)
    def M(self): self._rotate_slice(perspective=0, slice_idx=1, direction=-1)
    def m(self): self._rotate_slice(perspective=0, slice_idx=1, direction=1)
    def B(self): self._rotate_slice(perspective=0, slice_idx=2, direction=1)
    def b(self): self._rotate_slice(perspective=0, slice_idx=2, direction=-1)
    def U(self): self._rotate_slice(perspective=1, slice_idx=0, direction=-1)
    def u(self): self._rotate_slice(perspective=1, slice_idx=0, direction=1)
    def E(self): self._rotate_slice(perspective=1, slice_idx=1, direction=-1)
    def e(self): self._rotate_slice(perspective=1, slice_idx=1, direction=1)
    def D(self): self._rotate_slice(perspective=1, slice_idx=2, direction=1)
    def d(self): self._rotate_slice(perspective=1, slice_idx=2, direction=-1)
    def L(self): self._rotate_slice(perspective=2, slice_idx=0, direction=-1)
    def l(self): self._rotate_slice(perspective=2, slice_idx=0, direction=1)
    def S(self): self._rotate_slice(perspective=2, slice_idx=1, direction=-1)
    def s(self): self._rotate_slice(perspective=2, slice_idx=1, direction=1)
    def R(self): self._rotate_slice(perspective=2, slice_idx=2, direction=1)
    def r(self): self._rotate_slice(perspective=2, slice_idx=2, direction=-1)

    def apply_moves(self, move_sequence):
        if not isinstance(move_sequence, list) or not isinstance(move_sequence, str):
            raise ValueError("move_sequence must be a list of moves or a string of moves")
        for move in move_sequence:
                if move in self.move_map:
                    self.move_map[move]()
                else:
                    raise ValueError(f"Invalid move: {move}")

def get_piece_types(cube):
    """Identifies edge and corner piece IDs based on orientation markers."""
    corner_pieces = []
    edge_pieces = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                piece_id = cube.piece_current_positions[i, j, k]
                orientation = cube.piece_current_orientations[i, j, k]
                if orientation == 'x':  # Corner piece
                    corner_pieces.append(piece_id)
                elif orientation == 'g':  # Edge piece
                    edge_pieces.append(piece_id)
    return corner_pieces, edge_pieces

def calculate_distance_table(piece_ids, all_moves):
    """
    Calculates the minimum moves required to move one piece to another piece's position.
    """
    distance_table = {}
    solved_cube = Cube()  # Reference to solved state
    
    # Pre-calculate all piece positions in solved state
    solved_positions = {}
    for piece_id in piece_ids:
        solved_positions[piece_id] = solved_cube._get_position_of_piece(piece_id)
    
    for piece_id_start in piece_ids:
        for piece_id_target in piece_ids:
            if piece_id_start == piece_id_target:
                distance_table[(piece_id_start, piece_id_target)] = 0
                continue
                
            if (piece_id_target, piece_id_start) in distance_table:
                # Distance is symmetric for pieces of same type
                distance_table[(piece_id_start, piece_id_target)] = distance_table[(piece_id_target, piece_id_start)]
                continue
                
            # Position we want to move TO (where the first piece is in solved state)
            target_position = solved_positions[piece_id_start]
            
            # Initialize BFS
            queue = deque([(Cube(), 0)])  # (cube_state, move_count)
            visited = {cube_to_tuple(Cube())}
            
            while queue:
                current_cube, moves = queue.popleft()
                
                # Check if target piece has reached the desired position
                current_pos = current_cube._get_position_of_piece(piece_id_target)
                if current_pos == target_position:
                    distance_table[(piece_id_start, piece_id_target)] = moves
                    break
                
                # Safety limit to prevent infinite loops
                if moves >= 3:  # No piece should take more than 3 moves
                    break
                    
                # Try all possible moves
                for move in all_moves:
                    next_cube = copy.deepcopy(current_cube)
                    next_cube.move_map[move]()
                    
                    state_tuple = cube_to_tuple(next_cube)
                    if state_tuple not in visited:
                        visited.add(state_tuple)
                        queue.append((next_cube, moves + 1))
            
            if (piece_id_start, piece_id_target) not in distance_table:
                # If no path found, set to a high value or handle error
                distance_table[(piece_id_start, piece_id_target)] = -1
    
    return distance_table

def cube_to_tuple(cube):
    return tuple(cube.piece_current_positions.flatten().tolist())

def save_distance_table_to_json(distance_table, filename):
    serializable_table = {}
    for piece_pair, distance in distance_table.items():
        serializable_table[str(piece_pair)] = distance
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)


solved_cube = Cube()
corner_piece_ids, edge_piece_ids = get_piece_types(solved_cube)
all_moves = list(solved_cube.move_map.keys())

edge_distance_table = calculate_distance_table(edge_piece_ids, all_moves)
corner_distance_table = calculate_distance_table(corner_piece_ids, all_moves)

save_distance_table_to_json(edge_distance_table, "edge_distance_table.json")
save_distance_table_to_json(corner_distance_table, "corner_distance_table.json")

In [ ]:
from collections import deque
import numpy as np
import json
import copy

class Cube:
    def __init__(self):
        self.piece_current_positions = np.array([
                    [[1 , 2 , 3 ], [4 , 5 , 6 ], [7 , 8 , 9 ]],
                    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
                    [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
                ])
        #The PIECES are counted from Left-to-Right(axis=2), Top-to-Bottom (axis=1), and Front-to-Back (axis=0), in that order. The fourteenth piece is the invisible and irrelevant center-most piece of the cube

        self.piece_current_orientations = np.array([
            [['x', 'g', 'x'], ['g', 'F', 'g'], ['x', 'g', 'x']],
            [['g', 'U', 'g'], ['L', 'C', 'R'], ['g', 'D', 'g']],
            [['x', 'g', 'x'], ['g', 'B', 'g'], ['x', 'g', 'x']],
        ])
        # This is the intial state of the cube, which is a solved state
        # Notice that only the corners are marked by 'x' (other possible values are 'y' and 'z') and only the edges are marked by 'g' ('g' for good, other possible value is 'b' for bad)
        # Here x stands for the x-axis, and it means that the corner's fixed x-axis is aligned with the cube's x-axis (when the corner rotates, its axes rotate too) 
        # g stands for "good", which is one of the two possible orientations of an edge piece at any given position, and other one is b, which stands for "bad"
        # The center pieces are not marked, as they always remain the same

        self.move_map = {
            'U': self.U, 'F': self.F, 'B': self.B, 'D': self.D, 'L': self.L, 'R': self.R,
            'u': self.u, 'f': self.f, 'b': self.b, 'd': self.d, 'l': self.l, 'r': self.r,
            'M': self.M, 'E': self.E, 'S': self.S, 'm': self.m, 'e': self.e, 's': self.s,
        }

    def _rotate_slice(self, perspective, slice_idx, direction):
        # Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction
        def change_perspective(cube, perspective, direction):
            if perspective == 0: return cube
            else: return np.rot90(cube, k=direction, axes=(0, perspective))
        # Convert to the desired perspective, rotate the slice, then convert back
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        self.piece_current_positions[slice_idx] = np.rot90(self.piece_current_positions[slice_idx], k=direction, axes=(0, 1))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)

    def _get_position_of_piece(self, piece_id):
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_positions[i, j, k] == piece_id:
                        return (i, j, k)
        return None # Piece ID not found (should not happen in a valid cube state)
    
    def F(self): self._rotate_slice(perspective=0, slice_idx=0, direction=-1)
    def f(self): self._rotate_slice(perspective=0, slice_idx=0, direction=1)
    def M(self): self._rotate_slice(perspective=0, slice_idx=1, direction=-1)
    def m(self): self._rotate_slice(perspective=0, slice_idx=1, direction=1)
    def B(self): self._rotate_slice(perspective=0, slice_idx=2, direction=1)
    def b(self): self._rotate_slice(perspective=0, slice_idx=2, direction=-1)
    def U(self): self._rotate_slice(perspective=1, slice_idx=0, direction=-1)
    def u(self): self._rotate_slice(perspective=1, slice_idx=0, direction=1)
    def E(self): self._rotate_slice(perspective=1, slice_idx=1, direction=-1)
    def e(self): self._rotate_slice(perspective=1, slice_idx=1, direction=1)
    def D(self): self._rotate_slice(perspective=1, slice_idx=2, direction=1)
    def d(self): self._rotate_slice(perspective=1, slice_idx=2, direction=-1)
    def L(self): self._rotate_slice(perspective=2, slice_idx=0, direction=-1)
    def l(self): self._rotate_slice(perspective=2, slice_idx=0, direction=1)
    def S(self): self._rotate_slice(perspective=2, slice_idx=1, direction=-1)
    def s(self): self._rotate_slice(perspective=2, slice_idx=1, direction=1)
    def R(self): self._rotate_slice(perspective=2, slice_idx=2, direction=1)
    def r(self): self._rotate_slice(perspective=2, slice_idx=2, direction=-1)

    def apply_moves(self, move_sequence):
        if not isinstance(move_sequence, list) or not isinstance(move_sequence, str):
            raise ValueError("move_sequence must be a list of moves or a string of moves")
        for move in move_sequence:
                if move in self.move_map:
                    self.move_map[move]()
                else:
                    raise ValueError(f"Invalid move: {move}")

def get_piece_types(cube):
    """Identifies edge and corner pieces based on orientation markers."""
    corner_pieces = []
    edge_pieces = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                piece_id = cube.piece_current_positions[i, j, k]
                orientation = cube.piece_current_orientations[i, j, k]
                if orientation == 'x':  # corner
                    corner_pieces.append(piece_id)
                elif orientation == 'g':  # edge
                    edge_pieces.append(piece_id)
    return corner_pieces, edge_pieces

def build_single_piece_graph(all_moves, piece_type="edge"):
    """
    Precompute how each face turn affects a single piece's position
    (i, j, k). We do this for the solved cube, placing our 'test piece'
    in each valid position (corner or edge) and seeing where it moves.
    """
    # We only care about valid positions for the piece type
    valid_positions = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # Skip the center (1,1,1)
                if (i, j, k) == (1, 1, 1):
                    continue
                # Corner => 0 or 2 in all coords
                if piece_type == "corner":
                    if i in (0,2) and j in (0,2) and k in (0,2):
                        valid_positions.append((i,j,k))
                # Edge => exactly one coordinate == 1, others 0 or 2
                elif piece_type == "edge":
                    coords = (i, j, k)
                    if sum(c == 1 for c in coords) == 1 and all(c in (0,1,2) for c in coords):
                        valid_positions.append(coords)

    # Create a dictionary {position: {move: new_position}}
    # Then BFS will quickly find how many moves from start->end ignoring other pieces
    graph = {}
    solved = Cube()

    for pos in valid_positions:
        graph[pos] = {}
        # Place a "test piece" ID (e.g., 999) at pos in a copy of a solved cube
        # so we can track that "test piece" alone.
        test_cube = copy.deepcopy(solved)
        test_cube.piece_current_positions[pos] = 999  # place test piece at 'pos'
        original_id = solved.piece_current_positions[pos]

        # Temporarily remove the original piece ID so it doesn't conflict
        solved_index = np.argwhere(test_cube.piece_current_positions == original_id)
        for (i_s, j_s, k_s) in solved_index:
            test_cube.piece_current_positions[i_s, j_s, k_s] = -1  # placeholder

        # For each move, see where 999 goes
        for move in all_moves:
            temp_cube = copy.deepcopy(test_cube)
            temp_cube.move_map[move]()

            # Find the new position of 999
            new_pos = None
            for i_ in range(3):
                for j_ in range(3):
                    for k_ in range(3):
                        if temp_cube.piece_current_positions[i_, j_, k_] == 999:
                            new_pos = (i_, j_, k_)
                            break
                    if new_pos is not None:
                        break

            # Store the transition
            graph[pos][move] = new_pos

        # Restore original piece ID in the solved cube copy
        test_cube.piece_current_positions[pos] = original_id

    return graph

def calculate_distance_table_ignoring_others(piece_ids, all_moves, piece_type="edge"):
    """
    Builds a position graph for the piece type, then BFSes in that position graph
    from the start position to the target position. This ignores the rest of the puzzle.
    """
    # Build a small position graph for the relevant piece type
    pos_graph = build_single_piece_graph(all_moves, piece_type=piece_type)

    # Create a solved reference cube to look up positions
    solved_cube = Cube()

    distance_table = {}
    for pid_start in piece_ids:
        for pid_target in piece_ids:
            if pid_start == pid_target:
                distance_table[(pid_start, pid_target)] = 0
                continue

            # Start location of pid_start in the solved cube
            start_pos = solved_cube._get_position_of_piece(pid_start)
            # Target location of pid_target in the solved cube
            target_pos = solved_cube._get_position_of_piece(pid_target)

            # BFS in the position graph from start_pos to target_pos
            visited = set([start_pos])
            queue = deque([(start_pos, 0)])
            found_distance = -1

            while queue:
                current_pos, dist = queue.popleft()
                if current_pos == target_pos:
                    found_distance = dist
                    break
                # Explore all possible moves from current_pos in the graph
                for move in pos_graph[current_pos]:
                    next_pos = pos_graph[current_pos][move]
                    if next_pos not in visited:
                        visited.add(next_pos)
                        queue.append((next_pos, dist + 1))

            distance_table[(pid_start, pid_target)] = found_distance

    return distance_table

def save_distance_table_to_json(distance_table, filename):
    serializable_table = {}
    for pair, dist in distance_table.items():
        serializable_table[str(pair)] = dist
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)

def generate_move_table():
    """
    Generates a table mapping each move to the pieces it moves and their position changes.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              describing piece movements.
    """
    cube = Cube()  # Create a fresh cube in solved state
    movement_table = {}

    # Pieces that are not centers and thus their positions are tracked
    tracked_pieces = set(range(1, 28)) - {5, 11, 13, 14, 15, 17, 23}

    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube for each move calculation
        test_cube = Cube()

        # Record initial positions of tracked pieces
        initial_positions = {}
        for piece_id in tracked_pieces:
            initial_positions[piece_id] = test_cube._get_position_of_piece(piece_id)

        # Apply the move
        getattr(test_cube, move)()

        # Record final positions and calculate movements
        movements = {}
        for piece_id in tracked_pieces:
            initial_pos = initial_positions[piece_id]
            final_pos = test_cube._get_position_of_piece(piece_id)
            if initial_pos != final_pos:
                movements[piece_id] = {
                    "from": initial_pos,
                    "to": final_pos
                }

        # Store in table
        movement_table[move] = movements

    # Save to file
    with open('rubik_moves_table.json', 'w') as f:
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, piece_movements in movement_table.items():
            serializable_movements = {}
            for piece_id, movement in piece_movements.items():
                from_pos = ','.join(map(str, movement["from"]))
                to_pos = ','.join(map(str, movement["to"]))
                serializable_movements[piece_id] = {
                    "from": from_pos,
                    "to": to_pos
                }
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)

    return movement_table

if __name__ == "__main__":
    solved_cube = Cube()
    corner_piece_ids, edge_piece_ids = get_piece_types(solved_cube)
    all_moves = list(solved_cube.move_map.keys())
    for move in ['M', 'E', 'S', 'm', 'e', 's']:
        all_moves.remove(move)

    # Calculate corrected edge/corner distance tables ignoring the other pieces
    edge_distance_table = calculate_distance_table_ignoring_others(
        edge_piece_ids, all_moves, piece_type="edge"
    )
    corner_distance_table = calculate_distance_table_ignoring_others(
        corner_piece_ids, all_moves, piece_type="corner"
    )

    save_distance_table_to_json(edge_distance_table, "edge_distance_table.json")
    save_distance_table_to_json(corner_distance_table, "corner_distance_table.json")
    generate_move_table()

Loaded Edge Position Distance Table:
Distance between edge positions (0, 0, 1) and (0, 2, 1): 1


In [ ]:
from collections import deque
import numpy as np
import json
import copy

class Cube:
    def __init__(self):
        self.piece_current_positions = np.array([
                    [[1 , 2 , 3 ], [4 , 5 , 6 ], [7 , 8 , 9 ]],
                    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
                    [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
                ])
        #The PIECES are counted from Left-to-Right(axis=2), Top-to-Bottom (axis=1), and Front-to-Back (axis=0), in that order. The fourteenth piece is the invisible and irrelevant center-most piece of the cube

        self.piece_current_orientations = np.array([
            [['x', 'g', 'x'], ['g', 'F', 'g'], ['x', 'g', 'x']],
            [['g', 'U', 'g'], ['L', 'C', 'R'], ['g', 'D', 'g']],
            [['x', 'g', 'x'], ['g', 'B', 'g'], ['x', 'g', 'x']],
        ])
        # This is the intial state of the cube, which is a solved state
        # Notice that only the corners are marked by 'x' (other possible values are 'y' and 'z') and only the edges are marked by 'g' ('g' for good, other possible value is 'b' for bad)
        # Here x stands for the x-axis, and it means that the corner's fixed x-axis is aligned with the cube's x-axis (when the corner rotates, its axes rotate too) 
        # g stands for "good", which is one of the two possible orientations of an edge piece at any given position, and other one is b, which stands for "bad"
        # The center pieces are not marked, as they always remain the same

        self.move_map = {
            'U': self.U, 'F': self.F, 'B': self.B, 'D': self.D, 'L': self.L, 'R': self.R,
            'u': self.u, 'f': self.f, 'b': self.b, 'd': self.d, 'l': self.l, 'r': self.r,
            'M': self.M, 'E': self.E, 'S': self.S, 'm': self.m, 'e': self.e, 's': self.s,
        }

    def _rotate_slice(self, perspective, slice_idx, direction):
        """ Rotate a face (0=front, 1=middle, 2=back) seen from the given perspective (0=front, 1=top, 2=left) in the given direction """
        def change_perspective(cube, perspective, direction):
            if perspective == 0: return cube
            else: return np.rot90(cube, k=direction, axes=(0, perspective))
        # Convert to the desired perspective, rotate the slice, then convert back
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, -1)
        self.piece_current_positions[slice_idx] = np.rot90(self.piece_current_positions[slice_idx], k=direction, axes=(0, 1))
        self.piece_current_positions = change_perspective(self.piece_current_positions, perspective, 1)

    def _get_position_of_piece(self, piece_id):
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if self.piece_current_positions[i, j, k] == piece_id:
                        return (i, j, k)
        return None # Piece ID not found (should not happen in a valid cube state)
    
    def _get_piece_at_position(self, position):
        """Returns the piece ID at a given position (i, j, k)."""
        i, j, k = position
        return self.piece_current_positions[i, j, k]
    
    def F(self): self._rotate_slice(perspective=0, slice_idx=0, direction=-1)
    def f(self): self._rotate_slice(perspective=0, slice_idx=0, direction=1)
    def M(self): self._rotate_slice(perspective=0, slice_idx=1, direction=-1)
    def m(self): self._rotate_slice(perspective=0, slice_idx=1, direction=1)
    def B(self): self._rotate_slice(perspective=0, slice_idx=2, direction=1)
    def b(self): self._rotate_slice(perspective=0, slice_idx=2, direction=-1)

    def U(self): self._rotate_slice(perspective=1, slice_idx=0, direction=-1)
    def u(self): self._rotate_slice(perspective=1, slice_idx=0, direction=1)
    def E(self): self._rotate_slice(perspective=1, slice_idx=1, direction=-1)
    def e(self): self._rotate_slice(perspective=1, slice_idx=1, direction=1)
    def D(self): self._rotate_slice(perspective=1, slice_idx=2, direction=1)
    def d(self): self._rotate_slice(perspective=1, slice_idx=2, direction=-1)
    
    def L(self): self._rotate_slice(perspective=2, slice_idx=0, direction=-1)
    def l(self): self._rotate_slice(perspective=2, slice_idx=0, direction=1)
    def S(self): self._rotate_slice(perspective=2, slice_idx=1, direction=-1)
    def s(self): self._rotate_slice(perspective=2, slice_idx=1, direction=1)
    def R(self): self._rotate_slice(perspective=2, slice_idx=2, direction=1)
    def r(self): self._rotate_slice(perspective=2, slice_idx=2, direction=-1)

    def apply_moves(self, move_sequence):
        if not (isinstance(move_sequence, list) or isinstance(move_sequence, str)):
            raise ValueError("move_sequence must be a list of moves or a string of moves")
        for move in move_sequence:
                if move in self.move_map:
                    self.move_map[move]()
                else:
                    raise ValueError(f"Invalid move: {move}")

def get_piece_types(cube):
    """Identifies edge and corner pieces based on orientation markers."""
    corner_pieces = []
    edge_pieces = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                piece_id = cube.piece_current_positions[i, j, k]
                orientation = cube.piece_current_orientations[i, j, k]
                if orientation == 'x':  # corner
                    corner_pieces.append(piece_id)
                elif orientation == 'g':  # edge
                    edge_pieces.append(piece_id)
    return corner_pieces, edge_pieces

def build_single_piece_graph(all_moves, piece_type="edge"):
    """
    Precompute how each face turn affects a single piece's position
    (i, j, k). We do this for the solved cube, placing our 'test piece'
    in each valid position (corner or edge) and seeing where it moves.
    """
    # We only care about valid positions for the piece type
    valid_positions = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # Skip the center (1,1,1)
                if (i, j, k) == (1, 1, 1):
                    continue
                # Corner => 0 or 2 in all coords
                if piece_type == "corner":
                    if i in (0,2) and j in (0,2) and k in (0,2):
                        valid_positions.append((i,j,k))
                # Edge => exactly one coordinate == 1, others 0 or 2
                elif piece_type == "edge":
                    coords = (i, j, k)
                    if sum(c == 1 for c in coords) == 1 and all(c in (0,1,2) for c in coords):
                        valid_positions.append(coords)

    # Create a dictionary {position: {move: new_position}}
    # Then BFS will quickly find how many moves from start->end ignoring other pieces
    graph = {}
    solved = Cube()

    for pos in valid_positions:
        graph[pos] = {}
        # Place a "test piece" ID (e.g., 999) at pos in a copy of a solved cube
        # so we can track that "test piece" alone.
        test_cube = copy.deepcopy(solved)
        test_cube.piece_current_positions[pos] = 999  # place test piece at 'pos'
        original_id = solved.piece_current_positions[pos]

        # Temporarily remove the original piece ID so it doesn't conflict
        solved_index = np.argwhere(test_cube.piece_current_positions == original_id)
        for (i_s, j_s, k_s) in solved_index:
            test_cube.piece_current_positions[i_s, j_s, k_s] = -1  # placeholder

        # For each move, see where 999 goes
        for move in all_moves:
            temp_cube = copy.deepcopy(test_cube)
            temp_cube.move_map[move]()

            # Find the new position of 999
            new_pos = None
            for i_ in range(3):
                for j_ in range(3):
                    for k_ in range(3):
                        if temp_cube.piece_current_positions[i_, j_, k_] == 999:
                            new_pos = (i_, j_, k_)
                            break
                    if new_pos is not None:
                        break

            # Store the transition
            graph[pos][move] = new_pos

        # Restore original piece ID in the solved cube copy
        test_cube.piece_current_positions[pos] = original_id

    return graph

def calculate_distance_table_ignoring_others(piece_type="edge"): # Removed piece_ids arg, now using valid positions directly
    """
    Builds a position graph for the piece type, then BFSes in that position graph
    from the start position to the target position. This ignores the rest of the puzzle.
    The distance table is keyed by positions, not piece IDs.
    """
    solved_cube = Cube()
    all_moves = list(solved_cube.move_map.keys())
    core_moves = copy.deepcopy(all_moves)
    for move in ['M', 'E', 'S', 'm', 'e', 's']:
        core_moves.remove(move)
    pos_graph = build_single_piece_graph(core_moves, piece_type=piece_type) # Use core_moves here

    valid_positions = list(pos_graph.keys()) # Get valid positions directly from graph keys

    distance_table = {}
    for start_pos in valid_positions: # Iterate over positions, not piece_ids
        for target_pos in valid_positions:
            if start_pos == target_pos:
                distance_table[(start_pos, target_pos)] = 0
                continue

            if (target_pos, start_pos) in distance_table: #symmetry optimization
                distance_table[(start_pos, target_pos)] = distance_table[(target_pos, start_pos)]
                continue

            # BFS in the position graph from start_pos to target_pos
            visited = set([start_pos])
            queue = deque([(start_pos, 0)])
            found_distance = -1

            while queue:
                current_pos, dist = queue.popleft()
                if current_pos == target_pos:
                    found_distance = dist
                    break
                # Explore all possible moves from current_pos in the graph
                for move in pos_graph[current_pos]:
                    next_pos = pos_graph[current_pos][move]
                    if next_pos not in visited:
                        visited.add(next_pos)
                        queue.append((next_pos, dist + 1))

            distance_table[(start_pos, target_pos)] = found_distance

    return distance_table

def save_distance_table_to_json(distance_table, filename):
    serializable_table = {}
    for pos_pair, dist in distance_table.items(): # Key is now position pair
        serializable_table[str(pos_pair)] = dist
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)
        
def generate_move_table():
    """
    Generates a table mapping each move to the positions it affects and their new positions.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              mapping initial positions to final positions after the move.
    """
    cube = Cube()  # Create a fresh cube in solved state
    movement_table = {}

    # Positions that are not the center and thus their pieces' positions are tracked
    tracked_positions = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if (i, j, k) != (1, 1, 1):  # Exclude the center position
                    tracked_positions.append((i, j, k))

    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube for each move calculation
        test_cube = Cube()

        # Record initial piece IDs at each tracked position
        initial_piece_ids = {}
        for pos in tracked_positions:
            initial_piece_ids[pos] = test_cube._get_piece_at_position(pos)

        # Apply the move
        getattr(test_cube, move)()

        # Record final positions based on moved piece IDs
        movements = {}
        for initial_pos in tracked_positions:
            piece_id_to_track = initial_piece_ids[initial_pos]
            final_pos = test_cube._get_position_of_piece(piece_id_to_track) # Find where that piece ended up
            if initial_pos != final_pos:
                movements[initial_pos] = final_pos

        # Store in table
        movement_table[move] = movements

    # Save to file
    with open('rubik_position_moves_table.json', 'w') as f: # Changed filename
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, position_movements in movement_table.items():
            serializable_movements = {}
            for from_pos, to_pos in position_movements.items():
                from_pos_str = ','.join(map(str, from_pos))
                to_pos_str = ','.join(map(str, to_pos))
                serializable_movements[from_pos_str] = to_pos_str
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)

    return movement_table

# Example function to load the table (adjust to handle position keys)
def load_move_table():
    """Load the pre-generated move table with position keys"""
    with open('rubik_position_moves_table.json', 'r') as f: # Changed filename
        serialized_table = json.load(f)

    # Convert string positions back to tuples
    movement_table = {}
    for move, position_movements in serialized_table.items():
        movements = {}
        for from_pos_str, to_pos_str in position_movements.items():
            from_pos = tuple(map(int, from_pos_str.split(',')))
            to_pos = tuple(map(int, to_pos_str.split(',')))
            movements[from_pos] = to_pos
        movement_table[move] = movements

    return movement_table

def load_distance_table_from_json(filename):
    """Loads distance table, keys are now position tuples"""
    try:
        with open(filename, 'r') as f:
            serializable_table = json.load(f)
    except FileNotFoundError:
        print(f"Error: '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{filename}'. File may be corrupted.")
        return None

    distance_table = {}
    for pair_str, distance in serializable_table.items():
        pos_tuple = tuple(eval(pair_str)) # Use eval to convert string tuple back to tuple
        distance_table[pos_tuple] = distance
    return distance_table


# --- Example usage ---

if __name__ == "__main__":

    # Calculate corrected edge/corner distance tables ignoring the other pieces, keyed by position
    edge_distance_table = calculate_distance_table_ignoring_others(piece_type="edge")
    corner_distance_table = calculate_distance_table_ignoring_others(piece_type="corner")

    save_distance_table_to_json(edge_distance_table, "edge_position_distance_table.json") #saved with _position_ in filename
    save_distance_table_to_json(corner_distance_table, "corner_position_distance_table.json") #saved with _position_ in filename

    # Example of loading and retrieving by position
    loaded_edge_distance_table = load_distance_table_from_json("edge_position_distance_table.json") #load position-keyed table
    if loaded_edge_distance_table:
        print("Loaded Edge Position Distance Table:")
        # Example retrieval by position (get positions of pieces 2 and 8)
        solved_cube = Cube()
        pos1 = solved_cube._get_position_of_piece(2)
        pos2 = solved_cube._get_position_of_piece(8)
        distance = loaded_edge_distance_table.get((pos1, pos2), "Not found")
        print(f"Distance between edge positions {pos1} and {pos2}: {distance}")
    else:
        print("Edge position distance table not loaded.")

    generate_move_table()
    print(load_move_table())
    print(loaded_edge_distance_table)


Loaded Edge Position Distance Table:
Distance between edge positions (0, 0, 1) and (0, 2, 1): 2
{'U': {(0, 0, 0): (2, 0, 0), (0, 0, 1): (1, 0, 0), (0, 0, 2): (0, 0, 0), (1, 0, 0): (2, 0, 1), (1, 0, 2): (0, 0, 1), (2, 0, 0): (2, 0, 2), (2, 0, 1): (1, 0, 2), (2, 0, 2): (0, 0, 2)}, 'F': {(0, 0, 0): (0, 0, 2), (0, 0, 1): (0, 1, 2), (0, 0, 2): (0, 2, 2), (0, 1, 0): (0, 0, 1), (0, 1, 2): (0, 2, 1), (0, 2, 0): (0, 0, 0), (0, 2, 1): (0, 1, 0), (0, 2, 2): (0, 2, 0)}, 'B': {(2, 0, 0): (2, 2, 0), (2, 0, 1): (2, 1, 0), (2, 0, 2): (2, 0, 0), (2, 1, 0): (2, 2, 1), (2, 1, 2): (2, 0, 1), (2, 2, 0): (2, 2, 2), (2, 2, 1): (2, 1, 2), (2, 2, 2): (2, 0, 2)}, 'D': {(0, 2, 0): (0, 2, 2), (0, 2, 1): (1, 2, 2), (0, 2, 2): (2, 2, 2), (1, 2, 0): (0, 2, 1), (1, 2, 2): (2, 2, 1), (2, 2, 0): (0, 2, 0), (2, 2, 1): (1, 2, 0), (2, 2, 2): (2, 2, 0)}, 'L': {(0, 0, 0): (0, 2, 0), (0, 1, 0): (1, 2, 0), (0, 2, 0): (2, 2, 0), (1, 0, 0): (0, 1, 0), (1, 2, 0): (2, 1, 0), (2, 0, 0): (0, 0, 0), (2, 1, 0): (1, 0, 0), (2, 2, 0): 